# This notebook gives an instruction of how to load the pretrained model

## environment

In [ ]:
import torch
import numpy as np
import sys
import os
torch.set_default_dtype(torch.float64)
# setup gpu environment
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## get path

In [ ]:
path = os.getcwd() 
parent_path = os.path.abspath(os.path.join(path, os.pardir))
sys.path.append(parent_path)

## load pretrained models

In [ ]:
from Models.FDNET import fdnet
from Models.FDNET_FORCE import fdnet_force

# choose the case from the list
CASE = ['/stable/','/unstable/','/midnoise/','/force/','/fdblock/','/fdfilter/']

# taking the stable case as an example
case = CASE[0]

# choose networks
if case == 'force':
    DNN = 'fdnet_force'
else:
    DNN = 'fdnet'
    
xLen = 31 # spatial discretization
inputlength = 1 # short-term evolution interval, i.e., from t to t+\Delta t

# number of FD-Blocks{}
fb = 1
# numer of FD-Filters{}
fr = 16
# random seed, choose between 0 and 9
seed = 0


# construct the network model
model = getattr(sys.modules[__name__], DNN)(inputlength,fr,xLen).to(device)

# load the model paramters -- TR
algo = 'TR/'
TRweight = torch.load(path+case+algo+'Seed%s_BS%s_FR%s_FB%s_BEST.tar' %(seed,64,fr,fb))['weight']
for wi,mi in zip(TRweight,model.parameters()):
    mi.data.set_(torch.tensor(wi).to(device))
    
# load the model paramters -- ADAM of 1e-3
algo = 'Adam/'
A3weight = torch.load(path+case+algo+'Seed%s_BS%s_FR%s_FB%s_LR%s_BEST.tar' %(seed,64,fr,fb,1e-3))['weight']
for wi,mi in zip(A3weight,model.parameters()):
    mi.data.set_(torch.tensor(wi).to(device))
    
# load the model paramters -- ADAM of 1e-4
algo = 'Adam/'
A4weight = torch.load(path+case+algo+'Seed%s_BS%s_FR%s_FB%s_LR%s_BEST.tar' %(seed,64,fr,fb,1e-4))['weight']
for wi,mi in zip(A4weight,model.parameters()):
    mi.data.set_(torch.tensor(wi).to(device))



In [ ]:
exit(0)